- InfluxDB 入门安装教程 - 博客频道 - CSDN.NET http://t.cn/RS5ZVBJ
- python最易用的并发处理--multiprocessing.Pool进程池及ThreadPool线程池 http://t.cn/RS5Z5V9
- InfluxDB Python Examples — InfluxDB 4.1.0 documentation http://t.cn/RSz96IA
- python进程池multiprocessing.Pool和线程池multiprocessing.dummy.Pool实例 -  http://t.cn/RS5ZOtf
- Different Timestamps after uploading pandas Dataframe in InfluxDB · Issue #412 · influxdata/influxdb-python http://t.cn/RS5Zrix
- 多线程还是多进程的选择及区别 -   http://t.cn/RPXMTKc
- Convert dataframes directly to line protocol during 'write_points()' operation · Issue #363 · influxdata/influxdb-python http://t.cn/RS5y2uK

In [1]:
import tushare as ts
import pandas as pd
from datetime import datetime
from influxdb import InfluxDBClient
from influxdb import DataFrameClient
from multiprocessing import Pool  # 进程池
from multiprocessing.dummy import Pool as ThreadPool  # 线程池

In [2]:
class Tushare2Influxdb(object):
    
    def __init__(self):
        self.stocks_all=[]
        # 2 stocks first
        try:
            self.stocks_all=[i for i in ts.get_stock_basics().index][1596:]
#             self.stocks_all.append('600000')
        except:
            pass
        
        self._start_day='1997-01-01'
        self._today=datetime.today().strftime('%Y-%m-%d')
        self._ktype='D'
        self._measurement='stocks_all4'
        
        
    def get_dataframe(self,stock_code):
        try:
            data_frame=ts.get_k_data(stock_code,self._start_day)
            def dataframe2influxdb(dataframe):
                dataframe=dataframe.set_index('date')
                dataframe.index=pd.to_datetime(dataframe.index)
                dataframe['ktype']=self._ktype
                return dataframe
            return dataframe2influxdb(data_frame)
        except:
            return False
        

    def influxdf2db(self,influxdb_df_data):
        try:
            host='127.0.0.1'
            timeout=5
            database='tusharev1'
            client=DataFrameClient(host=host,timeout=timeout,database=database)
            client.create_database('tusharev1')
            measurement=self._measurement
            tag_columns=[ 'code', 'ktype']
            time_precision='s'
            protocol='json'
            batch_size=1000
            dataframe_to_influxdb=influxdb_df_data
            client.write_points(dataframe_to_influxdb,measurement,tag_columns=tag_columns,time_precision=time_precision,protocol=protocol,batch_size=batch_size)
        except:
            pass

        
    def do_it(self):
        for i in self.stocks_all:
            temp=self.get_dataframe(i)
            if temp.any().any() :
                self.influxdf2db(temp)
            else :
                pass
            
            
    def multithread(self,input_function,input_list):
        pool = ThreadPool(50)
        input_list=input_list
        input_function=input_function
        results = pool.map(input_function, input_list)
        pool.close() 
        pool.join()
        
    def do_it_multi(self):
        pool = ThreadPool(20)
        input_list=self.stocks_all
        def hyper_func(stock_code):
            try:
                return self.influxdf2db(self.get_dataframe(stock_code))
            except:
                return False
        input_function=hyper_func
        results = pool.map(hyper_func, input_list)
        pool.close() 
        pool.join()

s=Tushare2Influxdb()
s.do_it_multi()

# host='127.0.0.1'
# timeout=5
# database='tusharev1'
# client=DataFrameClient(host=host,timeout=timeout,database=database)
        

# d.get(list(d.keys())[0])

# x=pd.DataFrame({'a':[1,2],'b':[4,5]})

# x.any().any()



# s.get_dataframe('600000')

In [3]:
# import time
# tic=time.time()
# n=7000
# def m(x):
#     return x**x
# for i in range(n):
# #     i+=1
#     m(i)
# print(time.time()-tic)


In [4]:
# def multithread(input_function,input_list):
#     pool = ThreadPool(50)
#     input_list=input_list
#     input_function=input_function
#     results = pool.map(input_function, input_list)
#     pool.close() 
#     pool.join() 
#     results_wanted=[i for i in  results ]
#     return results_wanted

In [5]:

# import time
# tic=time.time()
# n=7000
# def m(x):
#     return x**x
 
# with ThreadPool(50) as pool:
#     pool.map(m,range(n))
     
    
# print(time.time()-tic)

In [6]:

client=DataFrameClient(timeout=None,database='tusharev1')

n=client.query('SHOW TAG VALUES FROM "stocks_all4" WITH KEY = "code"')

len([i for i in n.get_points()])

1594